<a href="https://colab.research.google.com/github/KaggleStudy4th/4th_kaggle_study/blob/main/%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C/12%EC%A3%BC%EC%B0%A8/%EB%AC%B8%EC%9E%90_%EB%8B%A8%EC%9C%84_RNN(Char_RNN)_%EB%8D%94_%EB%A7%8E%EC%9D%80_%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# 훈련데이터 전처리

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [3]:
char_set=list(set(sentence))
char_dic={c:i for i,c in enumerate(char_set)}

In [4]:
char_set

[' ',
 'l',
 'o',
 'd',
 ',',
 'm',
 's',
 'i',
 'f',
 'g',
 'h',
 'c',
 'b',
 '.',
 "'",
 'y',
 'u',
 'r',
 'a',
 'e',
 'k',
 't',
 'p',
 'w',
 'n']

In [5]:
char_dic

{' ': 0,
 "'": 14,
 ',': 4,
 '.': 13,
 'a': 18,
 'b': 12,
 'c': 11,
 'd': 3,
 'e': 19,
 'f': 8,
 'g': 9,
 'h': 10,
 'i': 7,
 'k': 20,
 'l': 1,
 'm': 5,
 'n': 24,
 'o': 2,
 'p': 22,
 'r': 17,
 's': 6,
 't': 21,
 'u': 16,
 'w': 23,
 'y': 15}

In [6]:
dic_size=len(char_dic)
print(dic_size)

25


In [7]:
#하이퍼파라미터 선택
hidden_size=dic_size
sequence_length=10# 임의 숫자 지정
learning_rate=0.1

임의로 지정한 sequence_length 값인 10의 단위로 샘플들을 잘라서 데이터를 만드는 모습을 보여줍니다.

In [8]:
#데이터 구성
x_data=[]
y_data=[]

for i in range(0,len(sentence)-sequence_length):
  x_str=sentence[i:i+sequence_length]
  y_str=sentence[i+1:i+sequence_length+1]
  print(i,x_str,'->',y_str)

  x_data.append([char_dic[c] for c in x_str])
  y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [9]:
print(x_data[0])
print(y_data[0])

[7, 8, 0, 15, 2, 16, 0, 23, 18, 24]
[8, 0, 15, 2, 16, 0, 23, 18, 24, 21]


In [11]:
import numpy as np

x_one_hot=[np.eye(dic_size)[x] for x in x_data] # x 데이터는 원-핫 인코딩
X=torch.FloatTensor(x_one_hot)
Y=torch.LongTensor(y_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  after removing the cwd from sys.path.


In [13]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [14]:
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [15]:
print(Y[0])

tensor([ 8,  0, 15,  2, 16,  0, 23, 18, 24, 21])


## 모델 구현

In [16]:
class Net(torch.nn.Module):
  def __init__(self,input_dim,hidden_dim,layers):
    super(Net,self).__init__()
    self.rnn=torch.nn.RNN(input_dim,hidden_dim,num_layers=layers,batch_first=True)
    self.fc=torch.nn.Linear(hidden_dim,hidden_dim,bias=True)

  def forward(self,x):
    x,_status=self.rnn(x)
    x=self.fc(x)
    return x

In [17]:
net=Net(dic_size,hidden_size,2)#층 두개 쌓기

In [22]:
criterion=torch.nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(),learning_rate)

In [19]:
outputs=net(X)
print(outputs.shape)

torch.Size([170, 10, 25])


(170, 10, 25)의 크기를 가지는데 각각 배치 차원, 시점(timesteps), 출력의 크기입니다. 나중에 정확도를 측정할 때는 이를 모두 펼쳐서 계산하게 되는데, 이때는 view를 사용하여 배치 차원과 시점 차원을 하나로 만듭니다

In [20]:
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환.

torch.Size([1700, 25])


In [21]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [24]:
for i in range(100):
  optimizer.zero_grad()
  outputs=net(X)# (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
  loss=criterion(outputs.view(-1,dic_size),Y.view(-1))
  loss.backward()
  optimizer.step()

  #results의 텐서 크기는 (170,10)
  results=outputs.argmax(dim=2)
  predict_str=""
  for j,result in enumerate(results):
    if j==0:
      predict_str+=''.join([char_set[t] for t in result])
    else:
      predict_str+=char_set[result[-1]]

  print(predict_str)

'rr'''rro''r'orro'''''o'o'''r'''o'o'''''''''o''''''o'r''r'''orro''''ooro''''o'''''o'o'oro''o''r'o''r''o'o''ro''''ro'r''''r'''o'''o'or'o'or'''oroo''''r'''''''ro'ooo'''''''oro'r''o'
odddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddd
ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
tuc,o l,csccnccnsc,c,nsn,cnotc,,cot,,non,ccnncccncn,,noosccn,cnso,cnncc,notn,ccnnnnn,,cnon,cn,cnno,cnnnn,ccc,cnnc,cnos,cnc,cn,,cntc,cncn,cnnoncnc,oon,cnsn,nnnnc,,ntnc,nccccnc,cnnn
d,o d,n  '   '  i    d,i   ii    i,n   '    i      ii ip'  k,p  ',  p,  i ,     ii      i,   i   k,  p,i  i i       i'   ,n  ,   ',     ', im   k  i,   p'   '     ,    i i  i,   i
dso meo mee  e  ed    eee  et     ee   t    e    meee  nee het  le   e    ee    ee       e       oe 